In [2]:
import mne
import sys
import os
import importlib

import braindecode
from braindecode.preprocessing.preprocess import preprocess, Preprocessor, zscore
from braindecode.datasets import (create_from_mne_raw, create_from_mne_epochs)
from braindecode.preprocessing.windowers import create_windows_from_events
from braindecode.datasets.sleep_physionet import SleepPhysionet
from braindecode.datasets import BaseDataset, BaseConcatDataset, WindowsDataset

from plot import Plot

'test'

'test'

In [24]:
raw_set = [
    '/home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf',
    '/home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf',
    '/home/maligan/mne_data/physionet-sleep-data/SC4441E0-PSG.edf',
    '/home/maligan/mne_data/physionet-sleep-data/SC4431E0-PSG.edf',
    '/home/maligan/mne_data/physionet-sleep-data/SC4421E0-PSG.edf',
]

# load into raw array
raws = [mne.io.read_raw_edf(x, preload=True) for x in raw_set]
# pick channels
raws = [x.pick_channels(['EEG Fpz-Cz', 'EEG Pz-Oz']) for x in raws]

Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8549999  =      0.000 ... 85499.990 secs...
Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8369999  =      0.000 ... 83699.990 secs...
Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4441E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7859999  =      0.000 ... 78599.990 secs...
Extracting EDF parameters from /home/maligan/mne_data/physionet-sleep-data/SC4431E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8429999  =      0.000 ... 84299.990 secs...
Extracting EDF parameters from /home/maligan/mne_data/ph

In [25]:
### preprocess
# resample to 160Hz
# high pass filtering of 30Hz

sfreq = 160
high_cut_hz = 30
n_jobs = 2

for raw in raws:
    # mne.io.Raw.resample(raw, sfreq, n_jobs=n_jobs) # resample all files
    mne.io.Raw.filter(raw, l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs) # high-pass filter

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    6.5s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.4s finished


In [26]:
base_datasets = [BaseDataset(raw) for raw in raws]

In [27]:
raws

[<RawEDF | SC4012E0-PSG.edf, 2 x 8550000 (85500.0 s), ~130.5 MB, data loaded>,
 <RawEDF | SC4451F0-PSG.edf, 2 x 8370000 (83700.0 s), ~127.7 MB, data loaded>,
 <RawEDF | SC4441E0-PSG.edf, 2 x 7860000 (78600.0 s), ~119.9 MB, data loaded>,
 <RawEDF | SC4431E0-PSG.edf, 2 x 8430000 (84300.0 s), ~128.6 MB, data loaded>,
 <RawEDF | SC4421E0-PSG.edf, 2 x 8292000 (82920.0 s), ~126.5 MB, data loaded>]

In [29]:
window_size_samples = 500

eegmmidb_windows = create_from_mne_raw(
    dataset,
    trial_start_offset_samples=0,
    trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples,
    drop_last_window=True,
    # descriptions=descriptions,
    accepted_bads_ratio=0.5,
    drop_bad_windows=True,
    # mapping=mapping,
    # preload=True
)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [17]:
# z-score normalization

preprocess(windows_dataset, Preprocessor(zscore))


<RawEDF | SC4451F0-PSG.edf, 7 x 13392000 (83700.0 s), ~715.2 MB, data loaded>

In [18]:
mne.io.RawArray(raws, mne.create_info())

TypeError: create_info() missing 2 required positional arguments: 'ch_names' and 'sfreq'